In [2]:
import os
import cv2
import glob
import torch
import shutil

import numpy as np
import pandas as pd
import torch.nn as nn
import albumentations as A
import torch.nn.functional as F
import matplotlib.pyplot as plt

from tqdm import tqdm
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from transformers import SegformerForSemanticSegmentation

/home/duong/anaconda3/envs/KLTN/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def find_bbox_from_mask(mask):
    """
    mask: numpy array (H, W)
    """
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    else:
        contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(contour)
        return x, y, w, h

def convert_bbox_size(x,y,w,h,mask_size, image_size, padding =(0,0)):
    """
    x,y,w,h: bbox size in mask
    mask_size: size of mask
    image_size: size of image
    """
    x_new = int((x*image_size[1])/mask_size[1]) - padding[0]
    y_new = int((y*image_size[0])/mask_size[0]) - padding[1]
    w_new = int((w*image_size[1])/mask_size[1]) + padding[0]*2
    h_new = int((h*image_size[0])/mask_size[0]) + padding[1]*2

    # check if bbox out of image
    if x_new < 0:
        x_new = 0
    if y_new < 0:
        y_new = 0

    if x_new + w_new > image_size[1]:
        w_new = image_size[1]
    if y_new + h_new > image_size[0]:
        h_new = image_size[0]

    return x_new, y_new, w_new, h_new

def check_image_new_size(w,h):
    # w,h << 50px -> not enough information
    if w == 0 or h == 0:
        return False
    if w < 50 or h < 50:
        return False
    #w = h*5 or h = w*5 -> not enough information
    elif w/h > 4 or h/w > 4:
        return False
    else:
        return True

def create_new_image(model, path_image, test_transform ,padding_percent, device, trainsize):
    image = cv2.imread(path_image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_original = image.copy()
    shape_original = image.shape
    image = test_transform(image=image)['image'].unsqueeze(0).to(device)
    image_mask_original = np.zeros((shape_original[0], shape_original[1]))

    with torch.no_grad():
        y_hat = model(image.to(device))
        y_hat = y_hat.logits
        y_hat =  F.interpolate(y_hat, size=(trainsize,trainsize), mode="bilinear").argmax(dim=1).squeeze().cpu().numpy()
        image_mask_original = cv2.resize(y_hat, (shape_original[1], shape_original[0]), interpolation=cv2.INTER_NEAREST)

    x, y, w, h = find_bbox_from_mask(y_hat)
    x1, y1, w1, h1 = convert_bbox_size(x,y,w,h,(trainsize,trainsize), \
                                                            shape_original,        \
                                                            padding=(0,0))
    image_orginal_bbox = cv2.rectangle(image_original, (x1,y1), (x1+w1,y1+h1), (255,0,0), 20)
    image_mask_original = image_mask_original*255
    image_mask_original_bbox = np.stack((image_mask_original,)*3, axis=-1)
    image_mask_original_bbox = cv2.rectangle(image_mask_original_bbox, (x1,y1), (x1+w1,y1+h1), (0,0,255), 20)
    x_img, y_img, w_img, h_img = convert_bbox_size(x,y,w,h,(trainsize,trainsize), \
                                                            shape_original,        \
                                                            padding=(int(padding_percent[0]*shape_original[0]/100),int( padding_percent[1]*shape_original[1]/100)))
    
    image_orginal_bbox = cv2.rectangle(image_original, (x_img,y_img), (x_img+w_img,y_img+h_img), (0,255,0), 20)
    image_mask_original_bbox = cv2.rectangle(image_mask_original_bbox, (x_img,y_img), (x_img+w_img,y_img+h_img), (0,255,0), 20)
    image_orginal_bbox = cv2.cvtColor(image_orginal_bbox, cv2.COLOR_RGB2BGR)
    name_image = path_image.split('/')[-1].split('.')[0]
    cv2.imwrite('Results/' +name_image+'_original_bbox.jpg', image_orginal_bbox)
    cv2.imwrite('Results/' +name_image+'_mask_original_bbox.jpg', image_mask_original_bbox)

    image = cv2.imread(path_image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    new_image = image[y_img:y_img+h_img, x_img:x_img+w_img]
    image_save = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite('Results/' +name_image+'_cropt.jpg', image_save)
                        
    if check_image_new_size(new_image.shape[1], new_image.shape[0]):
        return new_image, image_mask_original
    else:
        return image_original, image_mask_original

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
path_image = 'Data_demo/benign_skin/ISIC_0052212.jpg'
trainsize = 512
test_transform = A.Compose([
    A.Resize(width=trainsize, height=trainsize),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
    ToTensorV2(), # numpy.array -> torch.tensor (B, 3, H, W)
])
padding_percent = (20,20)
image_original, image_mask_original = create_new_image(model, path_image, test_transform ,padding_percent, device, trainsize)

In [3]:
from Training_ConvNeXt.utils.ConvNeXT_V1 import SkinClassifier

In [4]:
n_class = 2
model_classfication = SkinClassifier('convnext_small_in22ft1k',n_class)

/home/duong/anaconda3/envs/KLTN/lib/python3.8/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_small_in22ft1k to current convnext_small.fb_in22k_ft_in1k.
  model = create_fn(


In [7]:
model_classfication_path = 'model/convnext/model_epoch_4.pth'
if device == torch.device("cpu"):
    model_classfication_path = model_classfication_path.split('.')[0] + '_cpu.pth'
model_classfication.load_state_dict(torch.load(model_classfication_path))
model_classfication.to(device)
model_classfication.eval()

SkinClassifier(
  (backbone): ConvNeXt(
    (stem): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=96, out_features=384, bias=True)
              (act): GELU()
              (drop1): Dropout(p=0.0, inplace=False)
              (norm): Identity()
              (fc2): Linear(in_features=384, out_features=96, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (shortcut): Identity()
            (drop_path): Identity()
          )
          (1): ConvNeXtBlock(
            (conv_dw):

In [8]:
trainsize = 384
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)
test_transform_classfiation = A.Compose([
        A.Resize(width=trainsize, height=trainsize),
        A.Normalize(mean=mean, std=std, max_pixel_value=255),
        ToTensorV2(), # numpy.array -> torch.tensor (B, 3, H, W)
        ])

In [10]:
image_original_path = 'Data_demo/benign_skin/ISIC_2513338.jpg'
image_original = cv2.imread(image_original_path)
image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)
x = test_transform_classfiation(image=image_original)['image'].unsqueeze(0).to(device)

In [11]:
y_pred = model_classfication(x).float()  # Convert y_pred to float
probs = F.softmax(y_pred, dim=-1)
predictions = torch.argmax(probs, dim=-1)
predictions

tensor([0])